# 09 Geographic Proximity Dendrogram

This notebook maps each acute hospital in the **NHS South West** region to its closest Community Diagnostic Centre (CDC) and community hospital. The results are visualised as a dendrogram structured by Integrated Care Board (ICB).


## Load health infrastructure data

The input CSV files come from `data/processed/Health Infrastructure/enriched` and already contain latitude/longitude coordinates and ICB codes.

In [ ]:
import pandas as pd
import geopandas as gpd
import networkx as nx
from pyvis.network import Network


In [ ]:
acute_path = 'data/processed/Health Infrastructure/enriched/NHS_SW_Acute_Hospitals_enriched.csv'
cdc_path = 'data/processed/Health Infrastructure/enriched/NHS_SW_ Community_Diagnostic_Centres_enriched.csv'
comm_path = 'data/processed/Health Infrastructure/enriched/NHS_SW_Community_Hospitals_enriched.csv'

acute = pd.read_csv(acute_path)
cdc = pd.read_csv(cdc_path)
comm = pd.read_csv(comm_path)


In [ ]:
# convert to GeoDataFrames in British National Grid for distance calculation
acute_gdf = gpd.GeoDataFrame(
    acute,
    geometry=gpd.points_from_xy(acute.longitude, acute.latitude),
    crs='EPSG:4326'
).to_crs(27700)
cdc_gdf = gpd.GeoDataFrame(
    cdc,
    geometry=gpd.points_from_xy(cdc.longitude, cdc.latitude),
    crs='EPSG:4326'
).to_crs(27700)
comm_gdf = gpd.GeoDataFrame(
    comm,
    geometry=gpd.points_from_xy(comm.longitude, comm.latitude),
    crs='EPSG:4326'
).to_crs(27700)


In [ ]:
# find nearest CDC and community hospital for each acute site
out_cdc = gpd.sjoin_nearest(
    acute_gdf[['Code','Name','Parent Organisation Name','geometry']],
    cdc_gdf[['Code','Name','geometry']],
    how='left',
    distance_col='dist_cdc'
)
nearest_cdc = out_cdc.sort_values('dist_cdc').drop_duplicates('Code_left')

out_comm = gpd.sjoin_nearest(
    acute_gdf[['Code','Name','Parent Organisation Name','geometry']],
    comm_gdf[['Code','Name','geometry']],
    how='left',
    distance_col='dist_comm'
)
nearest_comm = out_comm.sort_values('dist_comm').drop_duplicates('Code_left')

summary = pd.DataFrame({
    'ICB': nearest_cdc['Parent Organisation Name'],
    'Acute Hospital': nearest_cdc['Name_left'],
    'Nearest CDC': nearest_cdc['Name_right'],
    'Nearest Community Hospital': nearest_comm.set_index('Code_left').loc[nearest_cdc['Code_left'], 'Name_right'].values
})
summary.head()


## Build dendrogram

A directed tree is created using NetworkX and visualised with pyvis.

In [ ]:
G = nx.DiGraph()
root = 'NHS South West'
G.add_node(root)
for _, row in summary.iterrows():
    icb = row['ICB']
    G.add_edge(root, icb)
    G.add_edge(icb, row['Acute Hospital'])
    G.add_edge(row['Acute Hospital'], f"CDC: {row['Nearest CDC']}")
    G.add_edge(row['Acute Hospital'], f"Community: {row['Nearest Community Hospital']}")

net = Network(height='700px', width='100%', directed=True)
net.from_nx(G)
net.show('graph.html')
